# Yolo FineTuning su dataset Linemood

Questo notebook permette di:
1. **Carica YOLO pretrained**
2. **Fine-tuning solo della head** (classification head)
3. **Inferenza sul Test Set e salvataggio metriche valutazione** 

## 1. Import e Setup

In [1]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


# Importa il config per usare path e file in altre cartelle
sys.path.insert(0, str(Path.cwd().parent))  # Aggiungi parent al path
from config import Config
from models.yolo_detector import YOLODetector
from models.yolo_detector import visualize_detections
from utils.prepare_yolo_symlinks import prepare_yolo_dataset_symlinks, create_data_yaml
from utils.organize_yolo_results import organize_yolo_output, print_organization_summary


# Usa PROJECT_ROOT dal config
project_root = Config.PROJECT_ROOT
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Configura matplotlib per notebook
%matplotlib inline

print(f"✅ Setup completato")
print(f"📂 Project root: {project_root}")
print(f"🎯 Classi LineMOD: {Config.NUM_CLASSES}")
print(f"🖥️  Device: {Config.DEVICE}")

Cache Strategy: full
Numworkers set to 0
Numworkers set to 0
✅ Setup completato
📂 Project root: /Users/nicolotermine/zMellow/GitHub-Poli/Polito/polito-aml-6D_pose_estimation
🎯 Classi LineMOD: 13
🖥️  Device: mps


In [2]:
detector = YOLODetector(
    model_name=Config.YOLO_MODEL,  # cambiare con nome di un modello già allenato per continuare da questo
    pretrained=True,
    num_classes=Config.NUM_CLASSES  # LineMOD ha 13 classi (la testa viene modificata automaticamente)
)

print(f"\n📊 Modello YOLO caricato:")
print(f"   Parametri totali: {detector.model_info['parameters']:,}")

# Mostra summary dettagliato del modello
print("\n📋 Model Summary (Layer Interni):")
print("="*80)
detector.model.info(verbose=True, detailed=False)  # verbose=True per vedere tutti i layer

✅ Loading pretrained yolo11n from cache: /Users/nicolotermine/zMellow/GitHub-Poli/Polito/polito-aml-6D_pose_estimation/checkpoints/pretrained/yolo11n.pt
⚠️  COCO pretrained weights have 80 classes, but you have 13 classes
🔄 Modifying detection head...
   ✅ Modified detection head: 80 → 13 classes
   📊 Output channels per scale: 18 (4 bbox + 1 obj + 13 classes)

📊 Modello YOLO caricato:
   Parametri totali: 2,624,080

📋 Model Summary (Layer Interni):
YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients


(181, 2624080, 0, 0.0)

In [3]:
output_dir = Config.LINEMOD_ROOT / 'yolo_symlinks'

# Prepara dataset se non esiste
if (output_dir / 'data.yaml').exists():
    print(f"✅ Dataset già preparato: {output_dir}")
    data_yaml_path = output_dir / 'data.yaml'
else:
    print("⚡ Preparazione dataset con symlinks...")
    stats = prepare_yolo_dataset_symlinks(
        dataset_root=Config.LINEMOD_ROOT,
        output_root=output_dir,
        use_symlinks=True
    )
    print(f"\n✅ Dataset preparato in {stats['time_seconds']:.1f}s")
    print(f"   Train: {stats['train']} immagini")
    print(f"   Val: {stats['val']} immagini")
    
    data_yaml_path = create_data_yaml(output_dir, Config.DATASETS_DIR)

print(f"\n📄 Config file: {data_yaml_path}")

✅ Dataset già preparato: /Users/nicolotermine/zMellow/GitHub-Poli/Polito/polito-aml-6D_pose_estimation/data/Linemod_preprocessed/yolo_symlinks

📄 Config file: /Users/nicolotermine/zMellow/GitHub-Poli/Polito/polito-aml-6D_pose_estimation/data/Linemod_preprocessed/yolo_symlinks/data.yaml


## 2. Fine-tuning solo della head

YOLO11n architecture (22 total layers, 0-21):
- Layer 0-9:   Backbone (feature extraction - CSPDarknet) 
- Layer 10-21: Neck + Head (feature fusion + detection)

ATTENZIONE: Yolo ha si 22 layer ma esplorando l'architettura se ne vedono molti di più (penso sia perchè sono blocchi di layer ripetuti)

In [4]:
# Freeze del backbone (mantieni solo head trainable)
print("❄️  Congelamento del backbone in corso...\n")

# Congela i primi 10 layer (backbone) per nuova classificazione
freeze_until = Config.YOLO_FREEZE_UNTIL_LAYER  
stats = detector.freeze_backbone(freeze_until_layer=freeze_until)

print(f"\n✅ Backbone congelato fino al layer {freeze_until}")
print(f"   Layer {freeze_until}-21 sono trainable (neck/head)")

# Visualizzazione compatta dei layer congelati
print("\n🔍 Stato Congelamento Layer:")
print("-" * 60)

for name, param in detector.model.named_parameters():
    status = "❄️  FROZEN" if not param.requires_grad else "🔥 TRAINABLE"
    # Estrai il numero del layer (es. model.10.conv.weight -> 10)
    layer_num = name.split('.')[1] if len(name.split('.')) > 1 else '?'
    print(f"Layer {layer_num:>2s} | {status} | {name[:50]:50s} | {param.numel():>10,} params")

print("-" * 60)

❄️  Congelamento del backbone in corso...

🔒 Freezing backbone until layer 10...

📊 Parameter Statistics:
   Total: 2,624,080
   Frozen: 1,115,744 (42.5%)
   Trainable: 1,508,336 (57.5%)

✅ Backbone frozen! Only detection head will be trained.

✅ Backbone congelato fino al layer 10
   Layer 10-21 sono trainable (neck/head)

🔍 Stato Congelamento Layer:
------------------------------------------------------------
Layer model | ❄️  FROZEN | model.model.0.conv.weight                          |        432 params
Layer model | ❄️  FROZEN | model.model.0.bn.weight                            |         16 params
Layer model | ❄️  FROZEN | model.model.0.bn.bias                              |         16 params
Layer model | ❄️  FROZEN | model.model.1.conv.weight                          |      4,608 params
Layer model | ❄️  FROZEN | model.model.1.bn.weight                            |         32 params
Layer model | ❄️  FROZEN | model.model.1.bn.bias                              |         32 para

In [5]:
# Training configuration (ora usa i parametri ottimizzati da Config)
EPOCHS = Config.YOLO_EPOCHS
BATCH_SIZE = Config.YOLO_BATCH_SIZE
IMG_SIZE = Config.YOLO_IMG_SIZE

# Run name
RUN_NAME = 'yolo_train50'
project_dir = Config.CHECKPOINT_DIR / 'yolo'
output_dir = project_dir / RUN_NAME

print(f"🔧 Parametri di Training:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Image size: {IMG_SIZE}")
print(f"   LR initial: {Config.YOLO_LR_INITIAL}")
print(f"   LR final: {Config.YOLO_LR_FINAL}")
print(f"   Optimizer: {Config.YOLO_OPTIMIZER}")
print(f"   Warmup epochs: {Config.YOLO_WARMUP_EPOCHS}")
print(f"   Cosine annealing: {Config.YOLO_COS_LR}")
print(f"   AMP: {Config.AMP_YOLO}")
print(f"   Num WORKERS: {Config.NUM_WORKERS}\n")

# Avvia training (ora i parametri ottimizzati sono defaults nel metodo train())
results = detector.train(
    data_yaml=str(data_yaml_path),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch_size=BATCH_SIZE,
    project=str(project_dir),
    name=RUN_NAME,
    save=True,
    cache=True,
    amp=Config.AMP_YOLO,  # ✅ Auto: True su CUDA (Windows), False su MPS (Mac)
    workers=Config.NUM_WORKERS,
    verbose=True
)

print(f"\n\n✅ Training completato!")
print(f"   Modello salvato in: {output_dir}")

# Organizza automaticamente i risultati in sottocartelle
print(f"\n📁 Organizzazione risultati...")
try:
    stats = organize_yolo_output(output_dir)
    print_organization_summary(output_dir, stats)
except Exception as e:
    print(f"⚠️  Errore nell'organizzazione: {e}")
    print(f"   I file sono comunque salvati in: {output_dir}")

🔧 Parametri di Training:
   Epochs: 50
   Batch size: 32
   Image size: 416
   LR initial: 0.01
   LR final: 0.0001
   Optimizer: SGD
   Warmup epochs: 3
   Cosine annealing: True
   AMP: False
   Num WORKERS: 0


🚂 Starting YOLO training...
   Model: yolo11n
   Epochs: 50
   Image size: 416
   Batch size: 32
   LR (initial → final): 0.01 → 0.0001
   Optimizer: SGD
   Device: mps
New https://pypi.org/project/ultralytics/8.3.238 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.234 🚀 Python-3.13.3 torch-2.9.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/Users/nicolotermine/zMellow/GitHub-Poli/Polito/polito-aml-6D_pose_estimation/data/Linemod_preprocessed/yolo_symlinks/data.yaml, degrees=0.0, deterministic=True, device=mps, df

KeyboardInterrupt: 

## 3. Load del modello Finetunato (inferenza e statistiche)

### Test su campione singolo

In [ ]:
# Carica il modello fine-tuned
YOLO_NAME = 'yolo_train10'
model_chechpoint_path = Config.CHECKPOINT_DIR / 'yolo' / YOLO_NAME
weights_path = model_chechpoint_path / 'weights' / 'best.pt'

if weights_path.exists():
    print(f"✅ Carico modello fine-tuned: {weights_path}\n")
    
    # Import necessari    
    finetuned_detector = YOLODetector(
        model_name=str(weights_path),
        num_classes=Config.NUM_CLASSES
    )
    
    # Carica immagine di test se non già caricata
    if 'image' not in locals():
        img_path = Config.LINEMOD_ROOT / 'data' / '01' / 'rgb' / '0000.png'
        image = np.array(Image.open(img_path).convert('RGB'))
        print(f"📷 Immagine caricata: {img_path}\n")
    else:
        # Se già caricata, mostra comunque il path
        img_path = Config.LINEMOD_ROOT / 'data' / '01' / 'rgb' / '0000.png'
        print(f"📷 Immagine già caricata: {img_path}\n")
    
    # Test sulla stessa immagine (usa Config.YOLO_CONF_THRESHOLD)
    detections_ft = finetuned_detector.detect_objects(image)
    
    # Mostra i nomi delle classi predette
    class_names_predetti = [det['class_name'] for det in detections_ft]
    print(f"🔎 Nomi classi predetti nella detection: {class_names_predetti}\n")
    
    print("\n📊 Visualizzazione detection...\n")
    
    # Visualizzazione
    vis_ft = visualize_detections(image.copy(), detections_ft)
    
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(vis_ft)
    plt.title(f"Fine-tuned LineMOD (Head only)\n{len(detections_ft)} detection", fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print(f"❌ Modello non trovato: {weights_path}")
    print("   Esegui prima il training (cella precedente)")

### Statistiche YOLO Fine Tunato

**Inferenza sul Test Set e salvataggio metriche valutazione** 

---
NOTA:

#### Validation sul Test Set LineMOD

Valutiamo le performance del modello fine-tuned sul test set ufficiale di LineMOD (mappato come "val" in YOLO).

**Note**: 
- Il "val" set è in realtà il **test set ufficiale** di LineMOD (~85% delle immagini)
- Il "train" set è molto piccolo (~15% delle immagini) seguendo la split ufficiale
- Metriche calcolate: mAP@0.5, mAP@0.5:0.95, Precision, Recall per classe

In [ ]:
# Validation del modello fine-tuned sul test set
YOLO_NAME = 'yolo_train10'
model_chechpoint_path = Config.CHECKPOINT_DIR / 'yolo' / YOLO_NAME
weights_path = model_chechpoint_path / 'weights' / 'best.pt'

if weights_path.exists() and 'data_yaml_path' in locals():
    print(f"📊 Validation del modello fine-tuned su test set LineMOD\n")
    print(f"   Modello: {weights_path.name}")
    print(f"   Dataset: {data_yaml_path}\n")
    print(f"⏳ Esecuzione validation (può richiedere alcuni minuti)...\n")
    
    # Carica modello se non già caricato
    if 'finetuned_detector' not in locals():
        finetuned_detector = YOLODetector(
            model_name=str(weights_path),
            num_classes=Config.NUM_CLASSES
        )
    
    # 🚀 OTTIMIZZAZIONE: conf=0.5 riduce drasticamente le detection candidate
    # Questo velocizza NMS di 5-10x (da ~18 min → ~2-3 min)
    print(f"⚙️  Confidence threshold: 0.5 (riduce overhead NMS)\n")
    
    # Esegui validation (ora usa i defaults da Config)
    metrics = finetuned_detector.validate(
        data_yaml=str(data_yaml_path),
        conf=Config.YOLO_CONF_THRESHOLD,  # ⚠️ CRITICO: riduce detection candidate per velocizzare NMS
        verbose=False
    )
    
    print(f"\n{'='*80}")
    print(f"📈 RISULTATI VALIDATION - YOLO11n Fine-tuned LineMOD")
    print(f"{'='*80}\n")
    
    # Metriche globali
    print(f"🎯 Metriche Globali:")
    print(f"   mAP@0.5:0.95:  {metrics.box.map:.4f}  (mean Average Precision)")
    print(f"   mAP@0.5:       {metrics.box.map50:.4f}  (AP @ IoU threshold 0.5)")
    print(f"   mAP@0.75:      {metrics.box.map75:.4f}  (AP @ IoU threshold 0.75)")
    print(f"   Precision:     {metrics.box.mp:.4f}")
    print(f"   Recall:        {metrics.box.mr:.4f}")
    
    # Metriche per classe
    print(f"\n📋 Metriche per Classe:")
    print(f"   {'Classe':<15} {'mAP@0.5':<10} {'mAP@0.5:0.95':<15} {'Precision':<12} {'Recall':<10}")
    print(f"   {'-'*70}")
    
    class_names = Config.CLASS_NAMES  # Lista dei nomi delle classi
    for i, class_name in enumerate(class_names):
        map50 = metrics.box.ap50[i] if i < len(metrics.box.ap50) else 0
        map_avg = metrics.box.ap[i] if i < len(metrics.box.ap) else 0
        prec = metrics.box.p[i] if i < len(metrics.box.p) else 0
        rec = metrics.box.r[i] if i < len(metrics.box.r) else 0
        
        print(f"   {class_name:<15} {map50:<10.4f} {map_avg:<15.4f} {prec:<12.4f} {rec:<10.4f}")
    
    # Statistiche aggiuntive
    print(f"\n📊 Statistiche Dataset:")
    print(f"   Immagini validate: ~{len(metrics.box.ap50) * 80} immagini")  # Stima
    print(f"   Classi: {Config.NUM_CLASSES}")
    print(f"   Image size: {Config.YOLO_IMG_SIZE}x{Config.YOLO_IMG_SIZE}")
    
    # Interpretazione risultati
    print(f"\n💡 Interpretazione:")
    if metrics.box.map50 > 0.8:
        print(f"   ✅ Ottimo! mAP@0.5 > 0.8 indica detection molto accurate")
    elif metrics.box.map50 > 0.6:
        print(f"   ✓  Buono! mAP@0.5 > 0.6 indica detection discrete")
    elif metrics.box.map50 > 0.4:
        print(f"   ⚠️  Discreto. mAP@0.5 > 0.4 ma potrebbe migliorare con più training")
    else:
        print(f"   ❌ Basso. Considerare più epoche o unfreezing di più layer")
    
    print(f"\n✅ Validation completata!")
    print(f"   Risultati salvati in: {model_chechpoint_path}")
    
else:
    if not weights_path.exists():
        print(f"❌ Modello non trovato: {weights_path}")
        print(f"   Esegui prima il training (cella 7)")
    else:
        print(f"❌ Dataset YAML non trovato")
        print(f"   Esegui prima la preparazione dataset (cella 4)")
